Creating a random graph with the help of random graph class which has been imported 
from other module. This random graph supports many other methods, some of them give information
about the graph and some do some kind of operations. Firstly creating graph using city data and then 
by processing we will create randomly connected graph which guaratee that it is connected but not 
fully.

In [1]:
# Importing Random_graph module
from Random_graph import Graph
import pandas as pd #required to read data from files.
import time
import sys

In [2]:
#Creating graph and generating randomly connected graph data.
df = pd.read_csv('Complete_graph.csv')

def create_graph():
    '''This returns the graph objects with unique nodes and edges created 
    from above read data from csv.'''
    G = Graph()
    G.add_nodes(df['node1'].unique())   #adding nodes
    for i, row in df.iterrows(): #adding edges iteratively.
        G.add_edge(row['node1'], row['node2'], row['route_distance'], row['Heuristic distance'])
    return G

def create_dropout_graph(graph, dropout):
    '''
    This function creates randomly connected graph after droping the percentage
    of edges asked by user to drop. There is maximum dropout beyond which user
    can not give dropout factor.Since we want connected graph so minimum dropout
    is 0 and maximum is given by below formula:
    Maximum Drouout = 1 - (Total number of unique nodes / Total number of edges)

    parameter graph: must be a graph instance.
    parameter dataframe: must be a valid dataframe.
    parameter dropout: must be valid dropout rate between 0 and max value.
    '''
    random_connected_graph = graph.randomGraphCreater(dropout)  #generating random graph.
    
    output_csv_file = 'random_connected_graph.csv'  
    edge_data = list(random_connected_graph)
    edge_df = pd.DataFrame(edge_data, columns=['node1', 'node2', 'route_distance', 'Heuristic distance'])
    edge_df.to_csv(output_csv_file, index=False)
    
    return f"Random connected graph saved to {output_csv_file}"

def create_random_graph_adjlist(graph):
    ''''''
    random_graph = graph._copySelf
    edges = random_graph.edges()
    dict = {}
    for edge in edges:  #iterating for each edge.
        if edge[0] not in dict:
            dict[edge[0]] = []
        if edge[0] != edge[1]:
            dict[edge[0]].append(edge[1]) #adding node to adjacency.
    return dict

def heuristic_root_distance(graph):
    '''Return a dictionary of for heuristic and root distances 
    between nodes.'''
    dict = {}   #heuristic root distances dictionary.
    edges = graph.edges()
    for edge in edges:
        if edge[0] not in dict:
            dict[edge[0]] = {}
        dict[edge[0]][edge[1]] = [edge[2], edge[3]]
    return dict

#--------------creating graph and droppped out graph---------------
graph = create_graph()
create_dropout_graph(graph, 0.95)
adj_list = create_random_graph_adjlist(graph)
heuristic_root_distances = heuristic_root_distance(graph)
#Information about graph.
print('Total number of nodes in graph: ', graph.total_nodes)
print('Total number of edges in graph: ', graph.total_edges)
print('Is graph connected: ', graph.is_connected())

Removing Edges and creating dropped out graph.: 1958it [00:50, 38.94it/s]                           

Total number of nodes in graph:  45
Total number of edges in graph:  101
Is graph connected:  True


Implementing informed and uninformed search strategies in following cells and testing them on above randomly connected graph and analysing them.

Implementing Depth first search an uninformed search algorithm.

In [3]:
start_time = time.perf_counter()
visited_list = []               #List to keep track of the nodes visited in the graph.
path = []                       #This stores the path followed to search for target node.
def DFS(starting_node, target_node):
    '''
    Search for target node and track the path of the target node from starting node.
    
    This will surely give path if target node exists and graph is connected.
    '''
    if starting_node not in visited_list:
        visited_list.append(starting_node)  #Appending node to visited list.
        if starting_node == target_node:    
            path.append(target_node)
            return True
        path.append(starting_node)
        for node in adj_list[starting_node]:
            if node not in visited_list:
                if DFS(node, target_node):
                    return True # Stopping further looping if target found
        path.pop()  #Removing element from path while backtracking.
    return False


#-------------Checking output of algorithm for different inputs-----------------
cases = {1: ['Bikaner', 'Patna'], 2: ['Sagar', 'Agra'], 3: ['Sikar', 'Gaya'], 4: ['Gaya', 'Sikar'], 5:['Mahoba', 'Pakur'], 6: ['Agra', 'Agra']}
memory = 0
total_time = 0
for case in cases:
    DFS(cases[case][0], cases[case][1])
    print(f'The path from {cases[case][0]} to {cases[case][1]} is: ', path)
    print('Path length is: ', len(path))
    print(f'Memory used for {cases[case][0]} to {cases[case][1]} is: ', sys.getsizeof(path) + sys.getsizeof(visited_list))
    print(f'time taken for {cases[case][0]} to {cases[case][1]} is: ', time.perf_counter() - start_time, '\n')
    start_time = time.perf_counter()
    total_time += start_time
    memory = memory + sys.getsizeof(path) + sys.getsizeof(visited_list)
    path = []
    visited_list = []
print("Total average time taken: ", total_time / 6)
print("Total average memory used: ", memory / 6, "bytes")

The path from Bikaner to Patna is:  ['Bikaner', 'Lucknow', 'Pakur', 'Palamu', 'Mahoba', 'Patna']
Path length is:  6
Memory used for Bikaner to Patna is:  592
time taken for Bikaner to Patna is:  0.0005769999988842756 

The path from Sagar to Agra is:  ['Sagar', 'Balaghat', 'Rajsamand', 'Chitrakoot', 'Sitamarhi', 'Delhi', 'Pakur', 'Lucknow', 'Kota', 'Agra']
Path length is:  10
Memory used for Sagar to Agra is:  432
time taken for Sagar to Agra is:  4.529999932856299e-05 

The path from Sikar to Gaya is:  ['Sikar', 'Mahoba', 'Palamu', 'Pakur', 'Lucknow', 'Kota', 'Agra', 'Gaya']
Path length is:  8
Memory used for Sikar to Gaya is:  592
time taken for Sikar to Gaya is:  7.050000203889795e-05 

The path from Gaya to Sikar is:  ['Gaya', 'Agra', 'Kota', 'Lucknow', 'Pakur', 'Palamu', 'Mahoba', 'Sikar']
Path length is:  8
Memory used for Gaya to Sikar is:  592
time taken for Gaya to Sikar is:  5.4299998737405986e-05 

The path from Mahoba to Pakur is:  ['Mahoba', 'Palamu', 'Pakur']
Path length 

Implementing Breadth first search an uninformed search algorithm.

In [4]:
#Implementing the Breadth first search algorithm.
start_time = time.perf_counter()
queue = []      #Queue used to keep all the nodes at any queue of graph.
visited_list = []       #Tracking the nodes which have been already visited
path = []       #Tracks path from starting to end
parent_child_mapping = {}       #Dictionary having child node as a key and parent node as value.
def BFS(starting_node, target_node):
    '''
    Traversing the graph to search for target node and generating path starting from 
    starting node.
    '''
    if starting_node not in visited_list:   #Visiting node if node is not visited.
        queue.append(starting_node)
        if starting_node == target_node:
            path_tracking(target_node)  #Generating path.
            return True
        for node in adj_list[starting_node]: #iterating for each node in graph.
            queue.append(node)
            # adding parent child relationship to dictionary.
            if node not in visited_list and node not in parent_child_mapping:
                parent_child_mapping[node] = starting_node
        queue.pop(0)
        visited_list.append(starting_node)
        BFS(queue[0], target_node)      #recursive definition.
    elif len(queue) != 0: #terminating if no more element in queue.
        queue.pop(0)
        if len(queue) != 0: #Performing recursion if more element in queue.
            BFS(queue[0], target_node)

def path_tracking(target_node):
    '''It is helper function to record path followed.'''
    node = target_node
    while node in parent_child_mapping: #Accessing the path.
        path.insert(0, node)
        node = parent_child_mapping[node]
    path.insert(0, node)


#-------------Checking output of algorithm for different inputs-----------------
cases = {1: ['Bikaner', 'Patna'], 2: ['Sagar', 'Agra'], 3: ['Sikar', 'Gaya'], 4: ['Gaya', 'Sikar'], 5:['Mahoba', 'Pakur'], 6: ['Agra', 'Agra']}
memory = 0
total_time = 0
for case in cases:
    BFS(cases[case][0], cases[case][1])
    print(f'The path from {cases[case][0]} to {cases[case][1]} is: ', path)
    print('Path length is: ', len(path))
    print(f'Memory used for {cases[case][0]} to {cases[case][1]} is: ', sys.getsizeof(path) + sys.getsizeof(visited_list))
    print(f'time taken for {cases[case][0]} to {cases[case][1]} is: ', time.perf_counter() - start_time, '\n')
    start_time = time.perf_counter()
    total_time += start_time
    memory = memory + sys.getsizeof(path) + sys.getsizeof(queue) + sys.getsizeof(visited_list)
    path = []
    visited_list = []
    queue = []
    parent_child_mapping = {}
print("Total average time taken: ", total_time / 6)
print("Total average memory used: ", memory / 6, "bytes")

The path from Bikaner to Patna is:  ['Bikaner', 'Lucknow', 'Pakur', 'Palamu', 'Mahoba', 'Patna']
Path length is:  6
Memory used for Bikaner to Patna is:  304
time taken for Bikaner to Patna is:  0.003605400001106318 

The path from Sagar to Agra is:  ['Sagar', 'Balaghat', 'Rajsamand', 'Chitrakoot', 'Sitamarhi', 'Delhi', 'Pakur', 'Lucknow', 'Kota', 'Agra']
Path length is:  10
Memory used for Sagar to Agra is:  560
time taken for Sagar to Agra is:  0.00019379999866941944 

The path from Sikar to Gaya is:  ['Sikar', 'Mahoba', 'Palamu', 'Pakur', 'Lucknow', 'Kota', 'Agra', 'Gaya']
Path length is:  8
Memory used for Sikar to Gaya is:  368
time taken for Sikar to Gaya is:  7.160000313888304e-05 

The path from Gaya to Sikar is:  ['Gaya', 'Agra', 'Kota', 'Lucknow', 'Pakur', 'Palamu', 'Mahoba', 'Sikar']
Path length is:  8
Memory used for Gaya to Sikar is:  304
time taken for Gaya to Sikar is:  4.989999797544442e-05 

The path from Mahoba to Pakur is:  ['Mahoba', 'Palamu', 'Pakur']
Path length i

Implementing Greedy best first search an informed search.

In [7]:
start_time = time.perf_counter()
path = []   # path of the target node
visited_list = [] # record for nodes visited.
def greedy_best_first_search(starting_node, target_node):
    if starting_node not in visited_list:
        visited_list.append(starting_node)
        if starting_node == target_node:
            path.append(target_node)
            return True
        path.append(starting_node)
        min = 10000000000000    #assuming all distances are lesser than this one.
        for node in adj_list[starting_node]:    
            hueristic_value = heuristic_root_distances[target_node][node][0] # searching for hueristic values.
            if min > hueristic_value:       # finding minimum ones.
                min = hueristic_value
                starting_node = node
        greedy_best_first_search(starting_node, target_node)        # recursion.


#--------------Checking output of algorithm for different startnode and goal node --------------
cases = {1: ['Bikaner', 'Patna'], 2: ['Sagar', 'Agra'], 3: ['Sikar', 'Gaya'], 4: ['Gaya', 'Sikar'], 5:['Bhopal', 'Morena'], 6: ['Agra', 'Agra']}
memory = 0
total_time = 0
for case in cases:
    greedy_best_first_search(cases[case][0], cases[case][1])
    if path[-1] != cases[case][1]:
        print(f'Could not found path from {cases[case][0]} to {cases[case][1]}.')
    else:
        print(f'The path from {cases[case][0]} to {cases[case][1]} is: ', path)
        print('Path length is: ', len(path))
    print(f'Memory used for {cases[case][0]} to {cases[case][1]} is: ', sys.getsizeof(path) + sys.getsizeof(visited_list))
    print(f'time taken for {cases[case][0]} to {cases[case][1]} is: ', time.perf_counter() - start_time, '\n')
    start_time = time.perf_counter()
    total_time += start_time
    memory = memory + sys.getsizeof(visited_list)  + sys.getsizeof(path)
    path = []
    visited_list = []
print("Total average time taken: ", total_time / 6)
print("Total average memory used: ", memory / 6, "bytes")

Could not found path from Bikaner to Patna.
Memory used for Bikaner to Patna is:  176
time taken for Bikaner to Patna is:  0.0007017999996605795 

Could not found path from Sagar to Agra.
Memory used for Sagar to Agra is:  176
time taken for Sagar to Agra is:  2.970000059576705e-05 

Could not found path from Sikar to Gaya.
Memory used for Sikar to Gaya is:  176
time taken for Sikar to Gaya is:  1.7300000763498247e-05 

Could not found path from Gaya to Sikar.
Memory used for Gaya to Sikar is:  176
time taken for Gaya to Sikar is:  1.6400001186411828e-05 

The path from Bhopal to Morena is:  ['Bhopal', 'Lakhimpur', 'Morena']
Path length is:  3
Memory used for Bhopal to Morena is:  176
time taken for Bhopal to Morena is:  2.4500001018168405e-05 

The path from Agra to Agra is:  ['Agra']
Path length is:  1
Memory used for Agra to Agra is:  176
time taken for Agra to Agra is:  1.300000076298602e-05 

Total average time taken:  22266.16050503333
Total average memory used:  176.0 bytes


Implementing A* search algorithm which is also a informed search strategy.

In [6]:
start_time = time.perf_counter()
visited_list = []           # nodes visited in tree.
path = []                   # path
parent_child_mapping = {}   # parent child relationship for nodes.
evaluated_value = {}        # the evaluation function value for nodes

def a_star_search(start_node, goal_node):
    '''Returns the path between start node and goal node.'''
    if start_node not in visited_list:
        visited_list.append(start_node)
        if start_node == goal_node:
            path_construct(goal_node)   #constructing path.
            return True
        for node in adj_list[start_node]:
            if node not in visited_list and node not in parent_child_mapping:
                parent_child_mapping[node] = start_node # creating parent child dictionary.
            if node not in visited_list:
                eval_value = heuristic_root_distances[node][goal_node][0] + heuristic_root_distances[visited_list[0]][node][1]
                evaluated_value[node] = eval_value  #finding values of evaluation function and storing.
        start_node = find_start_node()  #node for expansion with minimum evaluated value.
        del evaluated_value[start_node]
        a_star_search(start_node, goal_node)

def find_start_node():
    '''Returns  start node on the basis of minimum evaluated value 
    of evaluation of functions.'''
    min = 100000000000000   #assuming all distances are lesser than this.
    for key in evaluated_value:
        if evaluated_value[key] < min:
            min = evaluated_value[key]
            start_node = key
    return start_node

def path_construct(node):
    '''Return the path from stating node to goal node.'''
    node = node
    while node in parent_child_mapping: # iterating while there is parent for nodes.
        path.insert(0, node)
        node = parent_child_mapping[node]
    path.insert(0, node)


#------------checking output for different values of start node and goal node----------------------
cases = {1: ['Bikaner', 'Patna'], 2: ['Sagar', 'Agra'], 3: ['Sikar', 'Gaya'], 4: ['Gaya', 'Sikar'], 5:['Mahoba', 'Pakur'], 6: ['Agra', 'Agra']}
memory = 0
total_time = 0
for case in cases:
    a_star_search(cases[case][0], cases[case][1])
    print(f'The path from {cases[case][0]} to {cases[case][1]} is: ', path)
    print('Path length is: ', len(path))
    print(f'Memory used for {cases[case][0]} to {cases[case][1]} is: ', sys.getsizeof(path) 
          + sys.getsizeof(visited_list) + sys.getsizeof(parent_child_mapping) 
          + sys.getsizeof(parent_child_mapping))
    total_time += time.perf_counter() - start_time
    print(f'time taken for {cases[case][0]} to {cases[case][1]} is: ', time.perf_counter() - start_time, '\n')
    start_time = time.perf_counter()
    memory = memory + sys.getsizeof(path) + sys.getsizeof(visited_list) + sys.getsizeof(parent_child_mapping) + sys.getsizeof(parent_child_mapping)
    path = []
    visited_list = []
    parent_child_mapping = {}
    evaluated_value = {}
print("Total average time taken: ", total_time / 6)
print("Total average memory used: ", memory / 6, "bytes")

The path from Bikaner to Patna is:  ['Bikaner', 'Lucknow', 'Pakur', 'Palamu', 'Mahoba', 'Patna']
Path length is:  6
Memory used for Bikaner to Patna is:  1232
time taken for Bikaner to Patna is:  0.0012655999998969492 

The path from Sagar to Agra is:  ['Sagar', 'Balaghat', 'Rajsamand', 'Chitrakoot', 'Sitamarhi', 'Delhi', 'Pakur', 'Lucknow', 'Kota', 'Agra']
Path length is:  10
Memory used for Sagar to Agra is:  2096
time taken for Sagar to Agra is:  9.990000035031699e-05 

The path from Sikar to Gaya is:  ['Sikar', 'Mahoba', 'Palamu', 'Pakur', 'Lucknow', 'Kota', 'Agra', 'Gaya']
Path length is:  8
Memory used for Sikar to Gaya is:  1296
time taken for Sikar to Gaya is:  5.2299998060334474e-05 

The path from Gaya to Sikar is:  ['Gaya', 'Agra', 'Kota', 'Lucknow', 'Pakur', 'Palamu', 'Mahoba', 'Sikar']
Path length is:  8
Memory used for Gaya to Sikar is:  1232
time taken for Gaya to Sikar is:  3.590000051190145e-05 

The path from Mahoba to Pakur is:  ['Mahoba', 'Palamu', 'Pakur']
Path len